# Прогнозируем задержки самолетов

In [1]:
!pip install catboost lightgbm optuna -q

In [58]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import optuna

In [3]:
RANDOM_STATE = 111
DATASET_PATH = 'https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/flight_delays_train.csv'

In [4]:
data = pd.read_csv(DATASET_PATH)

X = data.drop('dep_delayed_15min', axis=1)
y = data['dep_delayed_15min'] == 'Y'

X.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732
1,c-4,c-20,c-3,1548,US,PIT,MCO,834
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423


Создайте список номеров колонок с категориальными признаками для бустингов

## Quiz
Какой длины получился список?

(подсказка: колонка `DepTime` числовая)

In [33]:
cat_features = np.where(X.dtypes==object)[0]

In [34]:
cat_features

array([0, 1, 2, 4, 5, 6])

Разобъем данные на обучение и контроль

In [19]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE)

In [20]:
Xtrain.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
41207,c-4,c-18,c-1,1457,CO,EWR,TPA,998
28283,c-11,c-1,c-2,1225,UA,DEN,BOS,1754
34619,c-6,c-16,c-5,1650,YV,IAD,CAE,401
8789,c-5,c-18,c-4,923,AA,SLC,DFW,988
38315,c-2,c-14,c-2,1839,AA,STL,SAN,1558


## Модели с параметрами по умолчанию

Обучите CatBoost с гиперпараметрами по умолчанию.

## Quiz
Чему равен ROC-AUC на тестовых данных? Ответ округлите до сотых.

In [ ]:
model = CatBoostClassifier(
    custom_loss=[metrics.Accuracy()],
    random_seed=42,
    logging_level='Silent'
)

In [36]:
model = CatBoostClassifier()
model.fit(Xtrain,ytrain,
          cat_features=cat_features,
          logging_level='Silent'
         )

In [37]:
y_pred=model.predict(Xtest)
y_pred_proba=model.predict_proba(Xtest)

In [43]:
y_pred_proba=y_pred_proba[:,1]

In [44]:
roc_auc_score(ytest,y_pred_proba)

0.7672794695374432

Обучите LightGBM с гиперпараметрами по умолчанию.

## Quiz
Чему равен ROC-AUC на тестовых данных? Ответ округлите до сотых.

In [49]:
for c in X.columns:
    col_type = X[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        Xtrain[c] = Xtrain[c].astype('category')
        Xtest[c] = Xtest[c].astype('category')

In [52]:
lgbm = LGBMClassifier()
lgbm.fit(Xtrain,ytrain)

y_pred=lgbm.predict(Xtest)
y_pred_proba=lgbm.predict_proba(Xtest)
y_pred_proba=y_pred_proba[:,1]

[LightGBM] [Info] Number of positive: 14346, number of negative: 60654
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002785 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 75000, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.191280 -> initscore=-1.441714
[LightGBM] [Info] Start training from score -1.441714


In [53]:
roc_auc_score(ytest,y_pred_proba)

0.7341149074685321

## Optuna

Выделим дополнительную валидационную выборку.

In [55]:
Xtrain_new, Xval, ytrain_new, yval = train_test_split(Xtrain, ytrain, test_size=0.25, random_state=RANDOM_STATE)

Создайте функцию objective_lgbm, в которой среди гиперпараметров

* num_leaves = trial.suggest_int("num_leaves", 10, 100)
* n_estimators = trial.suggest_int("n_estimators", 10, 1000)

подберите оптимальные, обучая LGBM на Xtrain_new, ytrain_new и проверяя качество (ROC-AUC) на Xval.

Используйте 30 эпох обучения Optuna.


In [60]:
def object(trial):
    num_leaves=trial.suggest_int('num_leaves',10,100)
    n_estimators=trial.suggest_int('n_estimators',10,1000)
    
    lgbm = LGBMClassifier(num_leaves=num_leaves,n_estimators=n_estimators)
    lgbm.fit(Xtrain_new,ytrain_new )
   
    y_pred=model.predict(Xval)
    y_pred_proba=model.predict_proba(Xval)
    y_pred_proba=y_pred_proba[:,0]
    
    score=roc_auc_score(yval , y_pred_proba)

    return score

study=optuna.create_study(direction='maximize')
study.optimize(object,n_trials=30)  
study.best_params

[I 2024-10-24 01:49:01,462] A new study created in memory with name: no-name-45d1f738-9b83-4516-8d57-da021c7eea36


[LightGBM] [Info] Number of positive: 10730, number of negative: 45520
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001199 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 56250, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.190756 -> initscore=-1.445108
[LightGBM] [Info] Start training from score -1.445108


[I 2024-10-24 01:49:03,222] Trial 0 finished with value: 0.1236333371731704 and parameters: {'num_leaves': 59, 'n_estimators': 532}. Best is trial 0 with value: 0.1236333371731704.


[LightGBM] [Info] Number of positive: 10730, number of negative: 45520
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001688 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 56250, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.190756 -> initscore=-1.445108
[LightGBM] [Info] Start training from score -1.445108


[I 2024-10-24 01:49:04,802] Trial 1 finished with value: 0.1236333371731704 and parameters: {'num_leaves': 15, 'n_estimators': 722}. Best is trial 0 with value: 0.1236333371731704.


...

[LightGBM] [Info] Number of positive: 10730, number of negative: 45520
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000398 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 56250, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.190756 -> initscore=-1.445108
[LightGBM] [Info] Start training from score -1.445108


[I 2024-10-24 01:49:48,581] Trial 28 finished with value: 0.1236333371731704 and parameters: {'num_leaves': 19, 'n_estimators': 528}. Best is trial 0 with value: 0.1236333371731704.


[LightGBM] [Info] Number of positive: 10730, number of negative: 45520
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000325 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 56250, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.190756 -> initscore=-1.445108
[LightGBM] [Info] Start training from score -1.445108


[I 2024-10-24 01:49:49,755] Trial 29 finished with value: 0.1236333371731704 and parameters: {'num_leaves': 29, 'n_estimators': 622}. Best is trial 0 with value: 0.1236333371731704.


{'num_leaves': 59, 'n_estimators': 532}

In [61]:
study.best_params

{'num_leaves': 59, 'n_estimators': 532}

Обучите модель с найденными гиперпараметрами на Xtrain, ytrain и оцените ROC-AUC на тестовых данных.

In [66]:
lgbm = LGBMClassifier()
lgbm.fit(Xtrain,ytrain)
   
y_pred=model.predict(Xtest)
y_pred_proba=model.predict_proba(Xtest)
y_pred_proba=y_pred_proba[:,1]
    
roc_auc_score(ytest , y_pred_proba)

[LightGBM] [Info] Number of positive: 14346, number of negative: 60654
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000444 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 75000, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.191280 -> initscore=-1.441714
[LightGBM] [Info] Start training from score -1.441714


0.7672794695374432

## Quiz

Чему равно количество деревьев в LGBM после подбора гиперпараметров?

## Работа над улучшением модели

* Попробуйте при помощи Optuna подобрать и другие гиперпарамтеры
* Также подберите гиперпараметры у CatBoost (а не только у LightGBM)

In [ ]:
# your code here

## Quiz

Поделитесь своими результатами!